<h1> Introdução </h1>

Maps nos permite transformar dados em um Dataframe ou Series um valor por vez em toda uma coluna. No entando, muitas vezes nós queremos agrupar nossos dados, e depois fazer algo específico com aquele grupo de dados.

Como nós iremos aprender, nós fazemos isso com o operador `groupby`. Nós tambéms vamos ver alguns tópicos adicionais, como uma maneira mais complexa de indexar seu DataFrame, e no final como  sequênciar seus dados.

In [3]:
import pandas as pd
df = pd.read_csv("winemag-data-130k-v2.csv", index_col=0)
pd.set_option("display.max_rows", 5)

<h1> Análise de Grupos inteligentes </h1>

Uma função que utilizamos bastante é `value_counts()`. Nós podemos reproduzir o que `value_counts()` faz com a seguinte célula abaixo:

In [12]:
df.groupby("points").points.count()

points
80     397
81     692
      ... 
99      33
100     19
Name: points, Length: 21, dtype: int64

`groupby()` cria um grupo de reviews que aloca os mesmos valores de pontos nos vinhos dados. Depois, para cada um desses grupos, nós pegamos a coluna `points()` e contamos quantas vezes cada um aparece.`value_counts()` é apenas um corta-caminho deste operador `groupby()`

Nós podemos utilizar qualquer função de resumo que utilizamos anteriormente com esses dados. Por exemplo, para pegar o vinho mais barato em cada categoria de pontos, podemos fazer o seguinte:

In [14]:
df.groupby("points").price.min()

points
80      5.0
81      5.0
       ... 
99     44.0
100    80.0
Name: price, Length: 21, dtype: float64

Você pode pensar que cada grupo que geramos é um pedaço do seu Dataframe contendo somente os valores que queremos. Esse Dataframe é accessivel para nós utilizando o método `apply()`, e podemos manipular os dados de qualquer maneira que já vimos.Por exemplo,abaixo você tem uma maneira de selecionar o nome do primeiro reviewer de cada vinho no dataset:

In [15]:
df.groupby("winery").apply(lambda dff: dff.title.iloc[0])

winery
1+1=3                          1+1=3 NV Rosé Sparkling (Cava)
10 Knots                 10 Knots 2010 Viognier (Paso Robles)
                                  ...                        
àMaurice    àMaurice 2013 Fred Estate Syrah (Walla Walla V...
Štoka                         Štoka 2009 Izbrani Teran (Kras)
Length: 16757, dtype: object

Para um refino ainda maior, você pode também agrupar mais de uma coluna. Por exemplo, aqui é como pegaríamos o melhor vinho de cada país e província. 

In [16]:
df.groupby(["country", "province"]).apply(lambda dff: dff.loc[dff.points.idxmax()])

country  \
country   province                      
Argentina Mendoza Province  Argentina   
          Other             Argentina   
...                               ...   
Uruguay   San Jose            Uruguay   
          Uruguay             Uruguay   

                                                                  description  \
country   province                                                              
Argentina Mendoza Province  If the color doesn't tell the full story, the ...   
          Other             Take note, this could be the best wine Colomé ...   
...                                                                       ...   
Uruguay   San Jose          Baked, sweet, heavy aromas turn earthy with ti...   
          Uruguay           Cherry and berry aromas are ripe, healthy and ...   

                                          designation  points  price  \
country   province                                                     
Argentina Mendoza Province           Nicasia Vineyard      97  120.0   
          Other                               Reserva      95   90.0   
...                                               ...     ...    ...   
Uruguay   San Jose           El Preciado Gran Reserva      87   50.0   
          Uruguay           Blend 002 Limited Edition      91   22.0   

                                    province region_1 region_2  \
country   province                                               
Argentina Mendoza Province  Mendoza Province  Mendoza      NaN   
          Other                        Other    Salta      NaN   
...                                      ...      ...      ...   
Uruguay   San Jose                  San Jose      NaN      NaN   
          Uruguay                    Uruguay      NaN      NaN   

                                  taster_name taster_twitter_handle  \
country   province                                                    
Argentina Mendoza Province  Michael Schachner           @wineschach   
          Other             Michael Schachner           @wineschach   
...                                       ...                   ...   
Uruguay   San Jose          Michael Schachner           @wineschach   
          Uruguay           Michael Schachner           @wineschach   

                                                                        title  \
country   province                                                              
Argentina Mendoza Province  Bodega Catena Zapata 2006 Nicasia Vineyard Mal...   
          Other                            Colomé 2010 Reserva Malbec (Salta)   
...                                                                       ...   
Uruguay   San Jose          Castillo Viejo 2005 El Preciado Gran Reserva R...   
          Uruguay           Narbona NV Blend 002 Limited Edition Tannat-Ca...   

                                          variety                winery  
country   province                                                       
Argentina Mendoza Province                 Malbec  Bodega Catena Zapata  
          Other                            Malbec                Colomé  
...                                           ...                   ...  
Uruguay   San Jose                      Red Blend        Castillo Viejo  
          Uruguay           Tannat-Cabernet Franc               Narbona  

[425 rows x 13 columns]

Outro método `groupby()` que vale a pena ser falado é `agg()`, que deixa você rodar diferentes funções no seu DataFrame simultaneamente. Por exxemplo, nós podemos gerar um simples resumo estatístico de um dataset como descrito abaixo: 

In [18]:
df.groupby(['country']).price.agg([len, min, max])

,len,min,max
country,,,
Argentina,3800.0,4.0,230.0
Armenia,2.0,14.0,15.0
...,...,...,...
Ukraine,14.0,6.0,13.0
Uruguay,109.0,10.0,130.0


Usar de maneira correta o `groupby()` permite que você faça coisas poderodas no seu dataset.


<h1>Multi-index</h1>

Em todos os exemplos que vimos até agora, trabalhamos apenas com um rótulo index. `groupby()` é um pouco diferente por conta disso, dependendo da operação que rodamos, algumas vezes o resultado saíra o que chamamos de multi-index.

Um multi-index difere do index normal pois possui diferentes níveis. Por exemplo:

In [20]:
countries_reviewed = df.groupby(["country", "province"]).description.agg([len])
countries_reviewed

len
country   province              
Argentina Mendoza Province  3264
          Other              536
...                          ...
Uruguay   San Jose             3
          Uruguay             24

[425 rows x 1 columns]

In [22]:
mi = countries_reviewed.index
type(mi)

pandas.core.indexes.multi.MultiIndex

Multiplos-indices tem inúmeros métodos para lidar com toda sua estrutura que é abstinta de indicies de nível único. Eles também dependem de dois níveis para retornar algum valor.

Nos casos que se usa multiplos-indices estão bem detalhados na documentação dos pandas avançado.

No entanto, no caso geral de multiplos índices o método que usaremos a maioria das vezes para fazer a reversão para os índices regulares é `reset_index()`

In [23]:
countries_reviewed.reset_index()

,country,province,len
0,Argentina,Mendoza Province,3264
1,Argentina,Other,536
...,...,...,...
423,Uruguay,San Jose,3
424,Uruguay,Uruguay,24


<h1> Ordenação </h1>

Olhando para `countries_reviewed` nós podemos ver que o grupo retorna os dados em uma ordem de indexamento, não por ordem de valor. Isso é dizer que, quanto ao resultado do groupby, a ordem das linhas dependem dos valores do index, não o valor dos dados. 

Para fazer os dados terem uma ordem nos podemos utilizar o método `sort_values()`.

In [25]:
countries_reviewed = countries_reviewed.reset_index()
countries_reviewed.sort_values(by='len')

,country,province,len
179,Greece,Muscat of Kefallonian,1
192,Greece,Sterea Ellada,1
...,...,...,...
415,US,Washington,8639
392,US,California,36247


Para ordernar pleo index, basta utilizar o método `sort_index()`

In [26]:
countries_reviewed.sort_index()

,country,province,len
0,Argentina,Mendoza Province,3264
1,Argentina,Other,536
...,...,...,...
423,Uruguay,San Jose,3
424,Uruguay,Uruguay,24


Finalmente, podemos ordenar mais que uma coluna por vez:

In [27]:
countries_reviewed.sort_values(by=['country', 'len'])

,country,province,len
1,Argentina,Other,536
0,Argentina,Mendoza Province,3264
...,...,...,...
424,Uruguay,Uruguay,24
419,Uruguay,Canelones,43


<h1>Exercícios</h1>

Rode o código abaixo:

In [1]:
import pandas as pd
df = pd.read_csv("winemag-data-130k-v2.csv", index_col=0)

## 1.
Quais são os revisiores de vinho que mais aparecem? Crie uma `Series` que indexa a categoria `taster_twitter_handle` do dataset e quantos reviews essa pessoa vez.

In [3]:
#Digite aqui sua reposta. Não esqueça de mostrar os resultados.

<details><summary>Clique aqui para ver a resposta.</summary>

```python
quantidade_reviews = df.groupby("taster_twitter_handle").taster_twitter_handle.count()
quantidade_reviews 
```    
</details>

## 2.
Qual é o melhor vinho que eu posso comprar dado determinada quantia de dinheiro? Crie uma `Series` no qual os index são os preços dos vinhos nos quais estão relacionados com seus pontos de review de forma crescente.

In [8]:
#Digite aqui sua reposta. Não esqueça de mostrar os resultados.

<details><summary>Clique aqui para ver a resposta.</summary>

```python
melhor_nota_por_preco = df.groupby("price")["points"].max().sort_index()
melhor_nota_por_preco
```     
</details>

## 3.
Quais são os minimos e máximos valores de preço por cada `variedade` de vinho? Crie um `DataFrame` no qual o index é a categoria `variety` do dataset e seus valores são o `min` e `max`.

In [12]:
#Digite aqui sua reposta. Não esqueça de mostrar os resultados.

<details><summary>Clique aqui para ver a resposta.</summary>

```python
preco_extremos = df.groupby("variety").price.agg([min, max])
preco_extremos
```     
</details>

## 4.
Qual a variedade de vinho mais cara? Crie uma variável `valores_ordenados` que contenha uma cópia do dataframe da questão anterior nos quais as variáveis estão organizadas de maneira descendente baseado no preço mínimo, depois no preço máximo

In [15]:
#Digite aqui sua reposta. Não esqueça de mostrar os resultados.

<details><summary>Clique aqui para ver a resposta.</summary>

```python
valores_ordenados = preco_extremos.sort_values(by=["min", "max"], ascending=False)
valores_ordenados
```     
</details>

## 5.
Crie uma `Series` no qual o index são os revisores e seus valores é a média dos pontos de cada review.

In [17]:
#Digite aqui sua reposta. Não esqueça de mostrar os resultados.

<details><summary>Clique aqui para ver a resposta.</summary>

```python
media_de_review = df.groupby("taster_name").points.mean()
media_de_review
```     
</details>

## 6.
Qual a combinação de países e variedades são as mais comuns? Crie uma `Series` no qual o index é um `MultiIndex` de `{country, variety}`. Organize os valores da `Series` de maneira decrescente baseado na contagem de vinhos.

In [ ]:
#Digite aqui sua reposta. Não esqueça de mostrar os resultados.

<details><summary>Clique aqui para ver a resposta.</summary>

```python
country_variety_counts = df.groupby(['country', 'variety']).size().sort_values(ascending=False)
country_variety_counts
```     
</details>